In [13]:
import argparse
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import json
from tqdm import tqdm

import librosa
import numpy as np
import torch
from transformers import pipeline

from panns_inference import AudioTagging, SoundEventDetection, labels


def find_audios(parent_dir, exts=['.wav', '.mp3', '.flac', '.webm', '.mp4', '.m4a']):
    audio_files = []
    for root, dirs, files in os.walk(parent_dir):
        for file in files:
            if os.path.splitext(file)[1] in exts:
                audio_files.append(os.path.join(root, file))
    return audio_files


#################### PANNs ####################

def load_panns(device='cuda'):
    model = AudioTagging(checkpoint_path=None, device=device)
    return model

@torch.no_grad()
def tag_audio(model, audio_path):
    (audio, _) = librosa.core.load(audio_path, sr=32000, mono=True)
    # only use the first 30 seconds
    audio = audio[None, :30*32000]
    (clipwise_output, embedding) = model.inference(audio)
    tags, probs = get_audio_tagging_result(clipwise_output[0])
    return tags, probs


def get_audio_tagging_result(clipwise_output):
    """Visualization of audio tagging result.
    Args:
      clipwise_output: (classes_num,)
    """
    sorted_indexes = np.argsort(clipwise_output)[::-1]

    tags = []
    probs = []
    for k in range(10):
        tag = np.array(labels)[sorted_indexes[k]]
        prob = clipwise_output[sorted_indexes[k]]
        tags.append(tag)
        probs.append(float(prob))

    return tags, probs 


def is_vocal(tags, probs, threshold=0.08):
    pos_tags = {'Speech', 'Singing', 'Rapping'}
    for tag, prob in zip(tags, probs):
        if tag in pos_tags and prob > threshold:
            return True
    return False


#################### Whisper ####################


def transcribe_and_save(whisper_model, panns_model, args):
    """transcribe the audio, and save the result with the same relative path in the output_dir
    """
    audio_files = find_audios(args.input_dir)

    if args.n_shard > 1:
        print(f'processing shard {args.shard_rank} of {args.n_shard}')
        audio_files.sort() # make sure no intersetction
        audio_files = audio_files[args.shard_rank * len(audio_files) // args.n_shard : (args.shard_rank + 1) * len(audio_files) // args.n_shard] 

    for file in tqdm(audio_files):
        output_file = os.path.join(args.output_dir, os.path.relpath(file, args.input_dir))
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        results = []
        try:
            tags, probs = tag_audio(panns_model, file)

            if args.threshold == 0. or is_vocal(tags, probs, threshold=args.threshold):
                if args.debug:
                    print(file)
                    for tag, prob in zip(tags, probs):
                        print(f'{tag}: {prob}')
                    continue

                ## generate 5 different transcription by varying the temperature
                result = whisper_model(file)
                results.append(result)

                with open(output_file + '.json', 'w') as f:
                    json.dump(results, f, indent=4, ensure_ascii=False)
            else:
                print(f'no vocal in {file}')
                if args.debug:
                    for tag, prob in zip(tags, probs):
                            print(f'{tag}: {prob}')
        except Exception as e:
            print(e)
            continue

In [14]:
class args:
    model = 'vinai/PhoWhisper-small' #vinai/PhoWhisper-large
    prompt = 'lyrics: '
    language = 'vi'
    input_dir = './sample'
    output_dir = './results'
    n_shard = 1
    shard_rank = 0
    threshold = 0
    debug = False
    top_n_sample = 2

In [15]:
panns_model = load_panns()

Checkpoint path: /home/anh/panns_data/Cnn14_mAP=0.431.pth


/home/anh/miniconda3/envs/music/lib/python3.8/site-packages/panns_inference/inference.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint

GPU number: 1


*ADDED PhoWhispers*

In [16]:
from transformers import pipeline
phowhisper_model = pipeline("automatic-speech-recognition", model=args.model, device="cuda", chunk_length_s=30)
# output = phowhisper_model('sample/en_sample1.mp3')['text']

/home/anh/.local/lib/python3.8/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [17]:
transcribe_and_save(phowhisper_model, panns_model, args)

  0%|          | 0/2 [00:00<?, ?it/s]/home/anh/.local/lib/python3.8/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
 50%|█████     | 1/2 [00:20<00:20, 20.21s/it]/home/anh/.local/lib/python3.8/site-packages/transformers/models/whisper/generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
100%|██████████| 2/2 [00:21<00:00, 10.54s/it]
